In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
players = pd.read_csv("Players.csv")
stats = pd.read_csv("Seasons_Stats.csv")

In [3]:
#players.head()
#stats.head()

In [4]:
df = stats.merge(players[["Player","height","weight","collage","born","birth_city","birth_state","birth_country"]], on = "Player", how="inner")

In [5]:
df["Year"] = pd.to_numeric(df["Year"])

In [6]:
df76 = df.loc[df.Year>=1976,]
df76= df76.loc[df76.Tm != "TOT",]

In [51]:
df76["birth_country"] = df76.birth_country.fillna("United States")
players["birth_country"] = players.birth_country.fillna("United States")

In [8]:
df76.loc[df.Pos == "PG-SG", 'Pos'] = "PG"
df76.loc[df.Pos == "SF-SG", 'Pos'] = "SF"
df76.loc[df.Pos == "C-PF", 'Pos'] = "C"
df76.loc[df.Pos == "SG-PG", 'Pos'] = "SG"
df76.loc[df.Pos == "PF-C", 'Pos'] = "PF"
df76.loc[df.Pos == "SG-SF", 'Pos'] = "SG"
df76.loc[df.Pos == "PF-SF", 'Pos'] = "PF"
df76.loc[df.Pos == "SF-PF", 'Pos'] = "SF"
df76.loc[df.Pos == "SG-SF", 'Pos'] = "SG"
df76.loc[df.Pos == "C-SF", 'Pos'] = "C"
df76.loc[df.Pos == "PG-SF", 'Pos'] = "PG"
df76.loc[df.Pos == "SG-PF", 'Pos'] = "SG"
# df76.Pos.value_counts()

In [9]:
# df76.groupby(["Year","Pos"]).mean()["height"].reset_index()
df76 =df76.drop(["Unnamed: 0","born"], axis=1)

In [10]:
df76["college_flag"] = df76.collage.notnull()

In [11]:
games = df76.groupby("Year").G.max()

In [12]:
games = np.minimum(82,games)

In [13]:
df76.columns = ['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2', 'OBPM', 'DBPM',
       'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'height', 'weight', 'college', 'birth_city',
       'birth_state', 'birth_country', 'college_flag']

Summation Statistics by team
FG, FGA, 3P, 3PA, 2P, 2PA, FT, FTA, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS

Average Statistics by team
Age, height, weight, went to college

In [14]:
team_sums = df76.groupby(["Year","Tm"]).sum().reset_index()[["Year","Tm","FG", "FGA", "3P",
                                                 "3PA", "2P", "2PA",
                                                 "FT", "FTA", "ORB",
                                                 "DRB", "TRB", "AST",
                                                 "STL", "BLK", "TOV", 
                                                 "PF", "PTS","college_flag"]]

In [15]:
team_sums_norm = team_sums.merge(games.to_frame().reset_index(), on="Year", how="inner")

In [16]:
team_sums_norm[["FG", "FGA", "3P","3PA", "2P", "2PA",
                          "FT", "FTA", "ORB", "DRB", "TRB", "AST",
                          "STL", "BLK", "TOV","PF", "PTS"]] = np.divide(team_sums_norm[["FG", "FGA", "3P","3PA", "2P", "2PA",
                          "FT", "FTA", "ORB", "DRB", "TRB", "AST",
                          "STL", "BLK", "TOV","PF", "PTS"]], np.transpose([team_sums_norm.G]*17))

In [17]:
team_sums_norm["FT%"] = team_sums_norm["FT"]/team_sums_norm["FTA"]
team_sums_norm["3P%"] = team_sums_norm["3P"]/team_sums_norm["3PA"]
team_sums_norm["2P%"] = team_sums_norm["2P"]/team_sums_norm["2PA"]

In [18]:
normalized_stats = team_sums_norm.groupby("Year").mean()

In [19]:
team_means = df76.groupby(["Year","Tm"]).mean().reset_index()[["Year","Tm","Age", "height", "weight"]]

In [20]:
positions_norm = df76.groupby(["Year","Pos"]).sum().reset_index()[["Year","Pos","G","FG", "FGA", "3P",
                                                 "3PA", "2P", "2PA",
                                                 "FT", "FTA", "ORB",
                                                 "DRB", "TRB", "AST",
                                                 "STL", "BLK", "TOV", 
                                                 "PF", "PTS","college_flag"]]

In [21]:
positions_norm[["FG", "FGA", "3P","3PA", "2P", "2PA",
                          "FT", "FTA", "ORB", "DRB", "TRB", "AST",
                          "STL", "BLK", "TOV","PF", "PTS"]] = np.divide(positions_norm[["FG", "FGA", "3P","3PA", "2P", "2PA",
                          "FT", "FTA", "ORB", "DRB", "TRB", "AST",
                          "STL", "BLK", "TOV","PF", "PTS"]], np.transpose([positions_norm.G]*17))

In [22]:
positions_norm_stats = positions_norm.drop("G",axis=1)

In [23]:
positions_norm_stats["FT%"] = positions_norm_stats["FT"]/positions_norm_stats["FTA"]
positions_norm_stats["3P%"] = positions_norm_stats["3P"]/positions_norm_stats["3PA"]
positions_norm_stats["2P%"] = positions_norm_stats["2P"]/positions_norm_stats["2PA"]

In [24]:
positions_means = df76.groupby(["Year","Pos"]).mean().reset_index()[["Year","Pos","Age", "height", "weight"]]

In [25]:
position_counts = df76.groupby(["Year","Pos"]).nunique()["Player"].to_frame().reset_index()

In [26]:
positions_means = positions_means.merge(position_counts, on = ["Year","Pos"], how="inner")

In [27]:
Joe_missings = df76.loc[df76.birth_city.isnull(),].groupby("Player").min()[["Year","college","birth_city","birth_state","birth_country"]]

In [52]:
globalization = df76.groupby(["Year","Player"]).mean().reset_index()[["Year","Player"]]

In [53]:
globalization = globalization.merge(players[["Player","birth_city","birth_state","birth_country"]], on = "Player", how= "left")

In [54]:
globalization

,Year,Player,birth_city,birth_state,birth_country
0,1976.0,Aaron James,New Orleans,Louisiana,United States
1,1976.0,Al Carlson,Oceanside,California,United States
2,1976.0,Al Eberhard,Cedar Rapids,Iowa,United States
3,1976.0,Alvan Adams,Lawrence,Kansas,United States
4,1976.0,Archie Clark,Conway,Arkansas,United States
5,1976.0,Austin Carr,Washington,District of Columbia,United States
6,1976.0,Barry Clemens,Dayton,Ohio,United States
7,1976.0,Bill Bradley*,Crystal City,Missouri,United States
8,1976.0,Bill Robinzine,Chicago,Illinois,United States
9,1976.0,Bill Walton*,La Mesa,California,United States


In [55]:
os.chdir("../")
positions_means.to_csv("Data/positions_means.csv")
normalized_stats.to_csv("Data/normalized_stats.csv")
positions_norm_stats.to_csv("Data/positions_norm_stats.csv")
team_sums_norm.to_csv("Data/team_sums_norm.csv")
team_means.to_csv("Data/team_means.csv")
Joe_missings.to_csv("Data/Joe_missings.csv")
globalization.to_csv("Data/globalization.csv")

In [81]:
'''
vizdf = df76.groupby(["Year"]).mean().reset_index()[["Year","3P","3PA","3P%","height","Age"]]
players_in_league = df76.groupby("Year").nunique()
plt.plot(players_in_league.index, players_in_league.Player)
#3-point attempts over time
plt.plot(vizdf.Year, vizdf["3PA"])
plt.plot(vizdf.Year, vizdf["3P"])
plt.plot(vizdf.Year, vizdf["3P%"])
plt.plot(vizdf.Year, vizdf["height"])
plt.plot(vizdf.Year, vizdf["Age"])
'''

'\nvizdf = df76.groupby(["Year"]).mean().reset_index()[["Year","3P","3PA","3P%","height","Age"]]\nplayers_in_league = df76.groupby("Year").nunique()\nplt.plot(players_in_league.index, players_in_league.Player)\n#3-point attempts over time\nplt.plot(vizdf.Year, vizdf["3PA"])\nplt.plot(vizdf.Year, vizdf["3P"])\nplt.plot(vizdf.Year, vizdf["3P%"])\nplt.plot(vizdf.Year, vizdf["height"])\nplt.plot(vizdf.Year, vizdf["Age"])\n'